# Systems and Transformations

In [1]:
# HIDE/SHOW
# ! pip install -U sympy 
# ! pip install -U numpy
# ! pip install -U matplotlib

# ! sudo apt install -y cm-super
# ! sudo apt install -y texlive texlive-latex-extra texlive-fonts-recommended dvipng

In [2]:
# HIDE/SHOW
# %reset -f
! rm -rfv *.png *.jpg
import os
import shutil
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML
import sympy as sp
import numpy as np
from lcapy import Circuit

from sympy.abc import t, x, y
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

saved_dir = "systems"
if os.path.exists(saved_dir):
    shutil.rmtree(saved_dir)
os.makedirs(saved_dir)

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Computer Modern Roman']
plt.rcParams['font.sans-serif'] = ['Computer Modern Sans Serif']
plt.rcParams['font.size'] = 24
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['axes.labelsize'] = 24
plt.rcParams['axes.titlesize'] = 24
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
%matplotlib inline 

In [3]:
# HIDE/SHOW            
def mpl_axes_plot(axes, expr, xrange, zero_is_none=False, **kwargs):
    x, start, stop = xrange
    func = sp.lambdify(x, expr)
    x = np.linspace(start, stop, 10000)
    y = func(x)
    axes.tick_params(axis='x')
    axes.tick_params(axis='y')
    axes.xaxis.set_major_locator(MaxNLocator(integer=True))
    axes.yaxis.set_major_locator(MaxNLocator(integer=True))
    axes.grid(False)
    axes.margins(0.02, 0.02)
    if zero_is_none:
        y[y==0] = np.nan
    axes.plot(x, y, **kwargs)
    axes.plot(x, np.zeros_like(x), color='k', alpha=0.5)
    return axes
    
def mpl_axes_stem(axes, expr, xrange, zero_is_none=False, **kwargs):
    x, start, stop, step = xrange
    func = sp.lambdify(x, expr, ['numpy', 'sympy'])
    x = np.arange(start, stop, step)
    y = func(x)
    axes.tick_params(axis='x')
    axes.tick_params(axis='y')
    axes.xaxis.set_major_locator(MaxNLocator(integer=True))
    axes.yaxis.set_major_locator(MaxNLocator(integer=True))
    axes.grid(False)
    axes.margins(0.02, 0.02)
    axes.plot(x, np.zeros_like(x), color='k', alpha=0.5)
    if zero_is_none:
        y[y==0] = np.nan
    axes.stem(x, y, basefmt=" ", use_line_collection=True, **kwargs)
    return axes
    
def mpl_axes_step(axes, expr, xrange, **kwargs):
    x, start, stop, step = xrange
    func = sp.lambdify(x, expr)
    x = np.arange(start, stop, step)
    y = func(x)
    axes.tick_params(axis='x')
    axes.tick_params(axis='y')
    axes.xaxis.set_major_locator(MaxNLocator(integer=True))
    axes.yaxis.set_major_locator(MaxNLocator(integer=True))
    axes.grid(False)
    axes.margins(0.02, 0.02)
    axes.plot(x, np.zeros_like(x), color='k', alpha=0.5)
    return axes.step(x, y, **kwargs)

## What are Systems?

- Systems are the mappings from input signals to output signals. 
- Systems are mathematical models that transform the input signals to the output signals. 
- Systems are mathematical models that transform single input signal to single output signal. 

## Types of Systems

- Continuous-Time (CT) Systems.
- Discrete-Time (CT) Systems.
- Deterministic Systems.
- Probabilistic Systems.

In [4]:
# HIDE/SHOW
Circuit("""
    S1 box; l=$x(t)$, right=0.5, aspect=1.1, draw=white
    S2 box; l=$h(t)$, right=0.5, aspect=1.1, draw=black
    S3 box; l=$y(t)$, right=0.5, aspect=1.1, draw=white
    W S1.e S2.w; right=0.5, endarrow=stealth, color=black
    W S2.e S3.w; right=0.5, endarrow=stealth, color=black
; label_nodes=false, scale=0.5
""").draw(os.path.join(saved_dir, "ht.png"))

Circuit("""
    S1 box; l=$x[n]$, right=0.5, aspect=1.1, draw=white
    S2 box; l=$h[n]$, right=0.5, aspect=1.1, draw=black
    S3 box; l=$y[n]$, right=0.5, aspect=1.1, draw=white
    W S1.e S2.w; right=0.5, endarrow=stealth, color=black
    W S2.e S3.w; right=0.5, endarrow=stealth, color=black
    ; label_nodes=false, scale=0.5
""").draw(os.path.join(saved_dir, "hn.png"))

Circuit("""
    S1 box; l=$x(t)$, down=0.5, aspect=1.1, draw=white
    S2 box; l=$h(t)$, down=0.5, aspect=1.1, draw=black
    S3 box; l=$y(t)$, down=0.5, aspect=1.1, draw=white
    W S1.e S2.w; down=0.5, endarrow=stealth, color=black
    W S2.e S3.w; down=0.5, endarrow=stealth, color=black
; label_nodes=false, scale=0.5
""").draw(os.path.join(saved_dir, "vt.png"))

Circuit("""
    S1 box; l=$x[n]$, down=0.5, aspect=1.1, draw=white
    S2 box; l=$h[n]$, down=0.5, aspect=1.1, draw=black
    S3 box; l=$y[n]$, down=0.5, aspect=1.1, draw=white
    W S1.e S2.w; down=0.5, endarrow=stealth, color=black
    W S2.e S3.w; down=0.5, endarrow=stealth, color=black
; label_nodes=false, scale=0.5
""").draw(os.path.join(saved_dir, "vn.png"))

<style>
td, th {
   border: none!important;
}
table th:first-of-type  {
    width: 50%;
}
table th:nth-of-type(2) {
    width: 50%;
}
img {
    display: block; 
    margin-left: auto;
    margin-right: auto;
}
</style>

## Block Diagrams

|      |      |
|:----:|:----:|
|![alt](systems/ht.png) | ![alt](systems/hn.png)    |
|                       |                           |
|                       |                           |
|![alt](systems/vt.png) | ![alt](systems/vn.png)    |